In [10]:
# import packages
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense, Dropout, SpatialDropout1D
from tensorflow.keras.layers import Embedding,BatchNormalization
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import string
nltk.download('punkt')
nltk.download('stopwords')
sns.set()

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/devarsh.patel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/devarsh.patel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
#import dataset and clean dataset
df = pd.read_csv('tweet_sentiment.csv')
df.drop_duplicates(inplace=True)
df.dropna(inplace=True)

In [12]:
#function to remove punctutaions 
def remove_punch(txt):
    txt_punch_removed = [char for char in txt if char not in string.punctuation ]
    return ''.join(txt_punch_removed)

In [13]:
#clena the text
df["cleaned_tweets"]=df["cleaned_tweets"].str.lower()
df["cleaned_tweets"] = df["cleaned_tweets"].apply(remove_punch)
stop_words = set(stopwords.words('english'))
df['cleaned_tweets'] = df['cleaned_tweets'].apply(lambda sent: [word for word in word_tokenize(sent) if word not in stop_words])
stemmer = PorterStemmer()
df['cleaned_tweets'] = df['cleaned_tweets'].apply(lambda words: [stemmer.stem(word) for word in words])

In [14]:
#dictionary to track the words from vocab
def get_dict(x:list):
    indx_count = 1
    word_to_indx = {}
    indx_to_word = {}
    for sent in x:
        for word in sent:
            if word not in word_to_indx.keys():
                word_to_indx[word] = indx_count
                indx_to_word[indx_count] = word
                indx_count +=1
    return word_to_indx, indx_to_word


In [15]:
word_to_indx, indx_to_word = get_dict(df["cleaned_tweets"])
volc_size = len(word_to_indx.keys())
volc_size += 1

In [16]:
import json 
       
      
# Serializing json  
with open("sample.json", "w") as outfile:
    json.dump(word_to_indx, outfile)

In [17]:
#converting textual data to numerical
input_size = 50
df["cleaned_tweets"] = df["cleaned_tweets"].apply(lambda words: [word_to_indx[word] for word in words])
data_x = pad_sequences(df["cleaned_tweets"], maxlen=input_size ,dtype="object", padding="post", truncating="post")
data_x = data_x.astype(np.float32)

In [18]:
data_y = pd.get_dummies(df["sentiment"],drop_first=False).values
data_y = data_y.astype(np.float32)

In [19]:
#split of data to train and test
X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.2)

In [25]:
#building model architecture
model = Sequential()
model.add(Embedding(volc_size, 32, input_length=input_size))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(2048,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(3,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
print(model.summary())

TypeError: add() got an unexpected keyword argument 'return_sequences'

In [21]:
#fitting model on train data
h = model.fit(X_train,y_train, validation_split=0.2, epochs=10, batch_size=32)

Epoch 1/10
440/440 [==============================] - 31s 64ms/step - loss: 1.0363 - accuracy: 0.5549 - val_loss: 0.9130 - val_accuracy: 0.5911
Epoch 2/10
440/440 [==============================] - 33s 75ms/step - loss: 0.9043 - accuracy: 0.5983 - val_loss: 0.9897 - val_accuracy: 0.3140
Epoch 3/10
440/440 [==============================] - 34s 78ms/step - loss: 0.8996 - accuracy: 0.6001 - val_loss: 0.9500 - val_accuracy: 0.5911
Epoch 4/10
440/440 [==============================] - 38s 86ms/step - loss: 0.8995 - accuracy: 0.5987 - val_loss: 0.9378 - val_accuracy: 0.5911
Epoch 5/10
440/440 [==============================] - 41s 93ms/step - loss: 0.8998 - accuracy: 0.6001 - val_loss: 1.0295 - val_accuracy: 0.5911
Epoch 6/10
440/440 [==============================] - 44s 100ms/step - loss: 0.8983 - accuracy: 0.6002 - val_loss: 0.9960 - val_accuracy: 0.5911
Epoch 7/10
440/440 [==============================] - 44s 101ms/step - loss: 0.8990 - accuracy: 0.6001 - val_loss: 0.9328 - val_accurac

In [22]:
#predict the sentiment on test
def pos_neg(y_pred):
  dec=''
  if np.argmax(y_pred)==0:
    dec='negative'
  elif np.argmax(y_pred)==1:
    dec='neutral'
  else:
    dec='positive'
  return dec


def predict(data):
  y_pred=model.predict(data)
  dec=pos_neg(y_pred)
  return dec

In [23]:
import pickle
pickle.dump(model, open('model.pkl', 'wb'))

INFO:tensorflow:Assets written to: ram://92c9dd54-6151-4e48-bf70-7d2dea3d14aa/assets


2/2 [==============================] - 0s 2ms/step


'negative'